In [ ]:
# ---------- Problem to solve ----------
# https://docs.python.org/3/library/dataclasses.html
# https://www.youtube.com/watch?v=vBH6GRJ1REM

# boiler plate code: This is the old fashion way, very labor intensive
class ManualComment:
    def __init__(self, id: int, text: str):
        self.__id: int = id
        self.__text: str = text
    
    @property   # make id read-only / immutable
    def id(self):
        return self.__id
    
    @property   # make text read-only / immutable
    def text(self):
        return self.__text

    def __repr__(self):
        return f"{self.__class__.__name__}(id={self.id}, text={self.text})"

    def __eq__(self, other):
        if other.__class__ is self.__class__:
            return (self.id, self.text) == (other.id, other.text)
        else:
            return NotImplemented
# NotImplemented signals to the runtime that it should ask someone else to satisfy the 
# operation. In the expression a == b, if a.__eq__(b) returns NotImplemented, 
# then Python tries b.__eq__(a). If b knows enough to return True or False, 
# then the expression can succeed. If it doesn't, then the runtime will fall back to the 
# built-in behavior (which is based on identity for == and !=)

    def __ne__(self, other):
        result = self.__eq__(other)
        if result is NotImplemented:
            return NotImplemented
        else:
            return not result

    def __hash__(self):
        return hash((self.__class__, self.id, self.text))

# from functools import total_ordering
# then you only have to define __eq__ and __lt__, and the rest will be automatically generated
    def __lt__(self, other):
        if other.__class__ is self.__class__:
            return (self.id, self.text) < (other.id, other.text)
        else:
            return NotImplemented

    def __le__(self, other):
        if other.__class__ is self.__class__:
            return (self.id, self.text) <= (other.id, other.text)
        else:
            return NotImplemented

    def __gt__(self, other):
        if other.__class__ is self.__class__:
            return (self.id, self.text) > (other.id, other.text)
        else:
            return NotImplemented

    def __ge__(self, other):
        if other.__class__ is self.__class__:
            return (self.id, self.text) >= (other.id, other.text)
        else:
            return NotImplemented


In [ ]:
# ---------- Explanation ----------
# Advantages of Dataclass over Namedtuples:
    # 1. Namedtuple is a tuple and all tuples can be compared to one another. 
    # so objects of different named tuple types could compare as equal if they have the same
    # number of members and same values for those numbers. Comparing objects of diff dataclasses
    # always returns False, as does comparing a dataclass object to a tuple object. 
    # 2. if you have code that unpacks a tuple, adding more members to that tuple breaks
    # the unpacking code. Dataclass objects cannot be unpacked, so you can add more
    # data attributes to a data class without breaking the existing code. 
    # 3. a dataclass can be a base class or a subclass in inheritance hierarchy. 
# Advantages over Tradional classes:
    # 1. data class autogenerates __init__, __repr__, __eq__
    # 2. data class can autogenerate overloaded <, >, comparison operators. 
    # 3. When you change data attributes defined in a dataclass, then use it in a script
    # or interactive session, the autogenerated code updates automatically. So you have
    # less code to maintain and debug. 
    # 4. The required variable annotations for class attributes and data attributes
    # enable you to take advantage of static code analysis tools so eliminate errors before
    # they can occur at execution time. 

In [2]:
# ---------- Dataclass, ex1 ----------
import dataclasses
from dataclasses import dataclass, field, astuple, asdict
import inspect
from pprint import pprint

# This is the dataclass way
# by default gives __eq__, __init__, __repr__
# frozen=True makes the class immutable, gives __hash__, __setattr__ (which is necessary to make it immutable)
# order=True gives __lt__, __le__, __gt__, __ge__
@dataclass(frozen=True, order=True)
class Comment:
    id: int
    text: str = ""  # default value
    # watch out for default mutable arguments, b/c every instance will share the same list
    # field(default_factory=list) will create a new list for each instance
    # in reality, text: str = "" is equal to text: str = field(default="")
    # compare=False will not include this field in the comparison
    # repr=False will not include this field in the __repr__ method
    # hash=False will not include this field in the __hash__ method
    replies: list[int] = dataclasses.field(default_factory=list, repr=False, compare=False)


comment = Comment(1, "I just subscribed!")
# comment.id = 3    # can't immutable
print(comment)      # Comment(id=1, text='I just subscribed!')      # call __repr__
print(dataclasses.astuple(comment))     # (1, 'I just subscribed!', [])
print(dataclasses.asdict(comment))      # {'id': 1, 'text': 'I just subscribed!', 'replies': []}
copy = dataclasses.replace(comment, id=3)   # Comment(id=3, text='I just subscribed!')
print(copy)     # Comment(id=3, text='I just subscribed!')

pprint(inspect.getmembers(Comment, inspect.isfunction))

Comment(id=1, text='I just subscribed!')
(1, 'I just subscribed!', [])
{'id': 1, 'text': 'I just subscribed!', 'replies': []}
Comment(id=3, text='I just subscribed!')
[('__delattr__', <function Comment.__delattr__ at 0x0000020EBB14E710>),
 ('__eq__', <function Comment.__eq__ at 0x0000020EBB14E3B0>),
 ('__ge__', <function Comment.__ge__ at 0x0000020EBB14E5F0>),
 ('__gt__', <function Comment.__gt__ at 0x0000020EBB14E560>),
 ('__hash__', <function Comment.__hash__ at 0x0000020EBB14E7A0>),
 ('__init__', <function Comment.__init__ at 0x0000020EBB14DFC0>),
 ('__le__', <function Comment.__le__ at 0x0000020EBB14E4D0>),
 ('__lt__', <function Comment.__lt__ at 0x0000020EBB14E440>),
 ('__repr__', <function Comment.__repr__ at 0x0000020EBB14C160>),
 ('__setattr__', <function Comment.__setattr__ at 0x0000020EBB14E680>)]


In [18]:
# ---------- Dataclass, class variable ----------
from dataclasses import dataclass
# this typing module used to declare ClassVar and List
from typing import ClassVar, List

@dataclass
class Card:
    # Below are arriable annotations
    # Class variables (belongs to the class, not individual instantiated object)
    FACES: ClassVar[List[str]] = ['Ace', '2', '3', '4', '5', '6', '7', 
                                  '8', '9', '10', 'Jack', 'Queen', 'King']
    SUITS: ClassVar[List[str]] = ['Hearts', 'Diamonds', 'Clubs', 'Spades']
    
    #data attributes (belongs to each object)
    face: str
    suit: str

    @property
    def image_name(self):
        """Return the Card's image file name."""
        return str(self).replace(' ', '_') + '.png'

    def __str__(self):
        """Return string representation for str()."""
        return f'{self.face} of {self.suit}'
    
    def __format__(self, format):
        """Return formatted string representation."""
        return f'{str(self):{format}}'

c1 = Card(Card.FACES[0], Card.SUITS[3])
c1              # Card(face='Ace', suit='Spades')
print(c1)       # Ace of Spades
c1.face         # 'Ace'
c1.image_name   # 'Ace_of_Spades.png'
c2 = Card(Card.FACES[0], Card.SUITS[3])
c2              # Card(face='Ace', suit='Spades')
c3 = Card(Card.FACES[0], Card.SUITS[0])
c3              # Card(face='Ace', suit='Hearts')
c1 == c2        # True
c1 == c3        # False
c1 != c3        # True

Ace of Spades


True

In [ ]:
# ---------- namedtuple ----------
# https://stackoverflow.com/questions/2970608/what-are-named-tuples-in-python
# https://www.youtube.com/watch?v=HGOBQPFzWKo&list=WL&index=7  - under collections chapter
# similar to struct in C++ - easy-to-create, lightweight object types
    # shorter code than defining a class manually
# immutable, backward compatible with tuples

from collections import namedtuple

# Ex 1:
Point = namedtuple('Point', 'x, y')     # Struct a class Point w/ attributes x, y
pt = Point(1, -4)   # instantiate Point class as pt object, w/ x=1, y=-4 attributes
print(pt)           # namedtuple has __repr__
# Point(x=1, y=-4)
print(pt.x, pt.y)   # can access object attributes like any other class
# 1 -4

# Ex 2:
Car = namedtuple('Car', 'color, mileage')

my_car = Car('red', 1000)
my_car.color
# 'red'
my_car.mileage
# 1000
my_car.color = 'blue'   # namedtuple like tuple is immutable 
# AttributeError: can't set attribute

li = ['green', 2000]
di = {'color': 'blue', 'mileage': 3000}

#using _make() to return namedtuple()
print(Car._make(li))    # make a new Car object from a sequence or iterable
# Car(color='green', mileage=2000)

# using ** operator to return namedtuple from dictionary
print(Car(**di))
# Car(color='blue', mileage=3000)

# using _asdict() to return an OrderedDict()
print(my_car._asdict())
# {'color': 'red', 'mileage': 3812.4}

In [6]:
# ---------- Enum ----------
from enum import Enum
class Day(Enum):
    MONDAY = 1
    TUESDAY = 2
    WEDNESDAY = 3

# print enum member
print(Day.MONDAY)                       # Day.MONDAY
# get name of memeber
print(Day.MONDAY.name)                  # MONDAY
# get value of member
print(Day.MONDAY.value)                 # 1
# get member by name
val = "MONDAY"
print(f'Value is {Day[val].value}')     # Value is 1

Day.MONDAY
MONDAY
1
Value is 1
